# Plot Viual Odometry

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import copy

def transform(t_x, t_y, t_z, r_x, r_y, r_z, r_w):
    return np.asarray(
        [[1-2*r_y**2-2*r_z**2, 2*r_x*r_y-2*r_z*r_w, 2*r_x*r_z+2*r_y*r_w, t_x],
         [2*r_x*r_y+2*r_z*r_w, 1-2*r_x**2-2*r_z**2, 2*r_y*r_z-2*r_x*r_w, t_y],
         [2*r_x*r_z-2*r_y*r_w, 2*r_y*r_z+2*r_x*r_w, 1-2*r_x**2-2*r_y**2, t_z],
         [0,0,0,1]])
# https://en.wikipedia.org/wiki/Rotation_matrix#General_rotations

In [2]:
df = pd.read_csv("/home/torsten/ms/citrus_data_set/04_13D_Jackal/vo.csv")
df

,sec,nanosec,t_x,t_y,t_z,r_x,r_y,r_z,r_w
0,1689715534,288268504,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1,1689715534,388132504,0.000124,0.000396,-0.000126,0.000195,0.000020,0.000047,1.0
2,1689715534,488175504,0.000190,0.000111,-0.000123,0.000083,-0.000051,-0.000018,1.0
3,1689715534,588150504,-0.000077,-0.000220,0.000158,0.000069,0.000016,0.000033,1.0
4,1689715534,688233504,-0.000069,-0.000262,0.000273,-0.000057,0.000038,-0.000053,1.0
...,...,...,...,...,...,...,...,...,...
7687,1689716302,987276504,0.000176,-0.000198,-0.000057,-0.000059,-0.000074,0.000055,1.0
7688,1689716303,87286504,0.000220,0.000245,0.000145,0.000088,-0.000030,0.000151,1.0
7689,1689716303,187281504,-0.000013,0.000146,0.000235,0.000016,0.000011,0.000127,1.0
7690,1689716303,287307504,-0.000005,-0.000063,0.000141,-0.000072,0.000026,0.000016,1.0


In [3]:
%matplotlib tk
plt.figure()
plt.plot(np.diff(df.iloc[:, 0]))
plt.show()

In [4]:
#df_kf = pd.read_csv("/home/torsten/ms/data/rosbag2_2023_11_07-15_41_39/kf.csv")
#df_kf = df_kf.iloc[1:]
#df_kf

In [5]:
poses = []
poses_kf = []
last_tf = transform(0,0,0,0,0,0,1)

vo_T_robot = np.asarray([[ 1,  0, 0, 0],
                         [ 0,  1, 0, 0],
                         [ 0,  0, 1, 0],
                         [ 0,  0, 0, 1]], dtype=float)

vTr_inv = np.linalg.inv(vo_T_robot)


for i, row in df.iterrows():
    tf = transform(row["t_x"], row["t_y"], row["t_z"], row["r_x"], row["r_y"], row["r_z"], row["r_w"])
    # Convert to robot frame
    tf = np.matmul(vo_T_robot, tf)
    tf = np.matmul(tf, vTr_inv)
    last_tf = np.matmul(last_tf, tf)
    poses.append(copy(last_tf))
"""
# Get transform of first keyframe
for index, row in df.iterrows():
    if row["timestamp"] >= df_kf.iloc[0, 0]:
        print(row["timestamp"])
        print(df_kf.iloc[0, 0])
        last_tf = poses[index]
        print(index)
        print(last_tf)
        break

#last_tf = transform(0,0,0,0,0,0,1)
for i, row in df_kf.iloc[1:].iterrows():
    tf = transform(row["t_x"], row["t_y"], row["t_z"], row["r_x"], row["r_y"], row["r_z"], row["r_w"])
    # Convert to robot frame
    tf = np.matmul(vo_T_robot, tf)
    tf = np.matmul(tf, vTr_inv)
    last_tf = np.matmul(last_tf, tf)
    poses_kf.append(copy(last_tf))
"""

    
print(last_tf)


    

[[ 8.06163984e-01 -3.72702505e-03  5.91680472e-01  9.08510221e+00]
 [-1.48342915e-01  9.66770041e-01  2.08206833e-01  3.10106569e+00]
 [-5.72794906e-01 -2.55620443e-01  7.78822311e-01  1.30734272e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [6]:
"""
poses.append(transform(0,0,0,0,0,0,1))

for i in range(3):
    tf =transform(0.2,0,0,0,0.707,0,0.707)
    last_tf = np.matmul(tf, last_tf)
    poses.append(copy(last_tf))

#poses.append(transform(2,0,0,0,0,0,1))
#poses.append(transform(3,0,0,0,0,0,1))
"""


'\nposes.append(transform(0,0,0,0,0,0,1))\n\nfor i in range(3):\n    tf =transform(0.2,0,0,0,0.707,0,0.707)\n    last_tf = np.matmul(tf, last_tf)\n    poses.append(copy(last_tf))\n\n#poses.append(transform(2,0,0,0,0,0,1))\n#poses.append(transform(3,0,0,0,0,0,1))\n'

In [7]:
poses = np.asarray(poses)
poses.shape

(7692, 4, 4)

In [8]:
poses_kf = np.asarray(poses_kf)
poses_kf.shape

(0,)

In [9]:
#"""
%matplotlib tk
plt.figure(figsize=(10,6))
plt.axis('equal')
plt.plot(poses[:, 0, 3], poses[:,2,3], "-x")
#plt.plot(poses_kf[:, 0, 3], poses_kf[:,1,3], "-x")
for i in range(0, poses.shape[0], 100):
    plt.text(poses[i,0,3], poses[i,2,3], str(i))
plt.show()
#"""

In [10]:
#df = pd.read_csv("gpsNED.csv")
#df

In [11]:
%matplotlib tk
plt.figure(figsize=(18,10))
plt.axis('equal')
plt.grid()
plt.title("Visual Odometry")
plt.scatter(df.iloc[:,3], -1*df.iloc[:,1])
plt.show()
